In [ ]:
import pandas as pd
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

# Load data into DataFrame
query = """ 
SELECT \"date\", sum(ammo_used) as data_field 
FROM battle_reports 
group by \"date\"
order by date;
"""
df = pd.read_sql(query, conn)

# Close connection
conn.close()


C:\Users\mykro\AppData\Local\Temp\ipykernel_40176\3365493932.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
df.head()

,date,data_field
0,2023-03-16,1632
1,2023-03-17,1742
2,2023-03-18,2547
3,2023-03-19,1487
4,2023-03-20,2526


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# ----------------------------
# Step 1: Load and prepare data
# ----------------------------

# Load your dataframe
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')
df.set_index('date', inplace=True)

data = df[['data_field']].values  # shape: (n_samples, 1)

# Normalize the data (LSTM works better with scaled data)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# ----------------------------
# Step 2: Create sequences
# ----------------------------

def create_sequences(data, look_back=7):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 7
X, y = create_sequences(data_scaled, look_back)

# Reshape X for LSTM: (samples, timesteps, features)
print("X shape:", X.shape)  # e.g. (samples, 7, 1)

# ----------------------------
# Step 3: Train/Test Split
# ----------------------------

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# ----------------------------
# Step 4: Build and train LSTM model
# ----------------------------

model = Sequential([
    LSTM(50, activation='relu', input_shape=(look_back, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# ----------------------------
# Step 5: Evaluate and save
# ----------------------------

loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")



X shape: (724, 7, 1)


c:\Users\mykro\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.2133
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0426
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0346
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0384
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0361
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0382
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0342
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0374
Epoch 9/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0345
Epoch 10/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0370
Epoch 11/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0353
Epoch 12/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0354
Epoch 13/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0353
Epoch 14/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0366
Epoch 15/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0347
Epoch 16/50
37/37 ━

In [7]:
import json
# Get the last known window
last_window = data_scaled[-look_back:].reshape(1, look_back, 1)

future_preds = []
current_input = last_window.copy()

for _ in range(7):
    pred = model.predict(current_input)[0]
    future_preds.append(pred[0])

    # Ensure pred is reshaped correctly to match (1, 1, 1)
    pred_reshaped = pred.reshape(1, 1, 1)

    # Concatenate along the time-step axis (axis=1)
    current_input = np.concatenate([current_input[:, 1:, :], pred_reshaped], axis=1)


# Inverse transform to original scale
future_preds_original = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))

# Build forecast with dates
last_date = df.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7)
forecast_df = pd.DataFrame({'date': forecast_dates, 'forecast': future_preds_original.flatten()})
print(forecast_df)

# Save to JSON
forecast_dict = dict(zip(forecast_df['date'].astype(str), forecast_df['forecast'].round(2)))
with open('forecast.json', 'w') as f:
    json.dump(forecast_dict, f, indent=4)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
        date     forecast
0 2025-03-16  2168.078857
1 2025-03-17  2191.371338
2 2025-03-18  2189.861328
3 2025-03-19  2207.353271
4 2025-03-20  2171.182861
5 2025-03-21  2153.736328
6 2025-03-22  2147.938721
